<a href="https://colab.research.google.com/github/FreeRikato/Slouch-Detection/blob/master/src/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - 2: Novelty in Data Preprocessing and ROI Segmentation

## **Todos**:

- [ ] Setup and Data collection
- [ ] Preprocessing
  - [ ] Pose Estimation
  - [ ] ROI Extraction
  - [ ] Background Subtraction
- [ ] Segmentation
  - [ ] Thresholding
  - [ ] Watershed Algorithm
- [ ] Comparison of Algorithms
- [ ] Visualization
- [ ] Analysis
  - [ ] Evaluation - Accuracy, Clarity and Computational Efficiency
  - [ ] Expected Findings - Thresholding and Watershed
- [ ] Conclusion

Real-time posture monitoring and correction systems are crucial for preventing **musculoskeletal disorders**, particularly in environments where prolonged sitting is common. Traditional methods of posture monitoring rely on **external sensors** or **manual observations**, which are often **intrusive** and **lack real-time feedback**. With advancements in computer vision, using **laptop cameras** to monitor posture offers a **non-intrusive**, **scalable** solution. However, existing techniques face challenges in accurately segmenting the Region of Interest (ROI) in varying **lighting conditions** and **complex backgrounds**.

## 1. Existing Methods and Limitation

* **Sensor-based Approaches**: These methods, such as those using **accelerometers** or **gyroscopes**, provide **high accuracy** but require additional **hardware**, making them less convenient and scalable.
* **Computer Vision-based Methods**: Approaches using **OpenPose** or MediaPipe for posture detection have shown promise, but they often struggle with accurately segmenting the ROI in diverse environments, particularly in the presence of **occlusions** or **non-uniform lighting**.

## 2. Proposed Method

### Rationale

The proposed method enhances the ROI by leveraging advanced preprocessing techniques combined with conventional segmentation algorithms. By focusing on the **key body parts** relevant to posture (e.g., shoulders, back, and head), the method aims to improve the **accuracy** and **reliability** of posture detection in real-time.

### Proposed Preprocessing Technique

* **Pose Estimation and ROI Enhancement**: Using OpenPose or MediaPipe to **identify and isolate key body joints**. The ROI is dynamically adjusted based on these key points, ensuring that only the relevant parts of the body are processed.
* **Background Subtraction**: Adaptive background subtraction is employed to **remove irrelevant background** elements, allowing for a cleaner segmentation of the body parts.

### Segmentation Algorithms

* **Thresholding**: A simple and fast technique that segments images based on **pixel intensity**. Useful for binary segmentation but may struggle with complex or non-uniform backgrounds.
* **Watershed Algorithm**: A region-based segmentation approach that treats the image as a **topographic surface**. Effective in separating **overlapping objects**, making it suitable for delineating different body parts within the ROI.

## 3. Implementation

### Setup and Data Collection

In [27]:
!pip install -qU opencv-python mediapipe numpy torch torchvision lib tensorflow tensorflow-lite

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow-lite (from versions: none)
ERROR: No matching distribution found for tensorflow-lite

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
import mediapipe as mp

In [1]:
# Initialize the video capture object
cap = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Display the frame
    cv2.imshow('Camera Feed', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture object and close windows
cap.release()
cv2.destroyAllWindows()


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/rikato/Code+Notes/Slouch-Detection/.venv/lib/python3.12/site-packages/cv2/qt/plugins"
QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot move to target thread (0x60b3b8ca36d0)

QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot move to target thread (0x60b3b8ca36d0)

QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot move to target thread (0x60b3b8ca36d0)

QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot move to target thread (0x60b3b8ca36d0)

QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot move to target thread (0x60b3b8ca36d0)

QObject::moveToThread: Current thread (0x60b3b8ca36d0) is not the object's thread (0x60b3b881d820).
Cannot

In [2]:
# Load the image
image = cv2.imread('image.jpg')
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Function to save the result
def save_result(img, model_name):
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Pose Estimation using {model_name}')
    plt.savefig(f'pose_estimation_{model_name}.jpg', bbox_inches='tight', pad_inches=0.1)
    plt.close()

### Preprocessing

#### Pose Estimation (media pipe and BlazePose)

##### Media Pipe

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Convert the BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the image and detect poses
        results = pose.process(image)
        
        # Draw pose annotations on the image
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        cv2.imshow('MediaPipe Pose', frame)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1725208114.682649 1881478 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725208114.685169 1883970 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.1.6-arch1.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1725208114.768732 1883959 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725208114.787709 1883956 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x624776d738f0) is not the object's thread (0x6247775fac60).
Cannot move to target thread (0x624776d738f0)

QObject::moveToThread: Current thread (0x624776d738f0) is not the object's thread (0x6247775fac60).
Cannot move to target thread (0x624776d738f0)

QObject::moveToThread: Current thread (0x624776d738f0) is not the object's t

In [4]:
# MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        annotated_image = image_rgb.copy()
        mp_drawing.draw_landmarks(
            annotated_image, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS
        )
        save_result(annotated_image, 'MediaPipe')


I0000 00:00:1725208118.750565 1881478 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725208118.757657 1884014 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.1.6-arch1.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1725208118.807983 1884011 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725208118.830743 1884000 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


##### Blaze Pose

In [1]:
# Initialize MediaPipe Pose
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize BlazePose model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        # Read frame from webcam
        success, frame = cap.read()
        if not success:
            print("Failed to read frame from webcam")
            break

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame for pose detection
        results = pose.process(frame_rgb)

        # Draw pose landmarks on the frame
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, 
                results.pose_landmarks, 
                mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
            )

        # Display the frame
        cv2.imshow('BlazePose Estimation', frame)

        # Exit loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

2024-09-01 21:56:52.810325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-01 21:56:52.824294: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-01 21:56:52.828162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-01 21:56:52.838315: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-01 21:56:53.624484: W tensorflow/compiler/tf2

In [4]:
# BlazePose (using MediaPipe)
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

with mp_pose.Pose(static_image_mode=True, model_complexity=2, min_detection_confidence=0.5) as pose:
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        annotated_image = image_rgb.copy()
        mp_drawing.draw_landmarks(
            annotated_image, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
        )
        print("Saved")
        save_result(annotated_image, 'BlazePose')

I0000 00:00:1725205681.803377 1834363 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725205681.804204 1835763 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.1.6-arch1.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1725205681.859666 1835753 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725205681.921203 1835751 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [23]:
import cv2
import mediapipe as mp
import numpy as np
import os
from datetime import datetime

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize webcam
cap = cv2.VideoCapture(0)

# Create a directory to save images
save_dir = "saved_images"
os.makedirs(save_dir, exist_ok=True)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image and detect pose
        results = pose.process(image)

        # Convert back to BGR for OpenCV
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        roi = None  # Initialize roi variable

        if results.pose_landmarks:
            # Draw pose landmarks
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Get landmarks for head and shoulders
            landmarks = results.pose_landmarks.landmark
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
            nose = landmarks[mp_pose.PoseLandmark.NOSE.value]

            # Calculate ROI coordinates
            h, w, _ = image.shape
            x_min = int(min(left_shoulder.x, right_shoulder.x, nose.x) * w)
            x_max = int(max(left_shoulder.x, right_shoulder.x, nose.x) * w)
            y_min = int(min(left_shoulder.y, right_shoulder.y, nose.y) * h)
            y_max = int(max(left_shoulder.y, right_shoulder.y, nose.y) * h)

            # Add padding to ROI
            padding = 20
            x_min = max(0, x_min - padding)
            x_max = min(w, x_max + padding)
            y_min = max(0, y_min - padding)
            y_max = min(h, y_max + padding)

            # Draw ROI rectangle
            cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Extract ROI
            roi = image[y_min:y_max, x_min:x_max]

            # Display ROI in a separate window
            cv2.imshow('ROI', roi)

        # Display the full image
        cv2.imshow('MediaPipe Pose', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('s'):
            # Generate timestamp for unique filenames
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            
            # Save full image
            full_image_path = os.path.join(save_dir, f"full_image_{timestamp}.jpg")
            cv2.imwrite(full_image_path, image)
            print(f"Full image saved: {full_image_path}")

            # Save ROI image if available
            if roi is not None:
                roi_image_path = os.path.join(save_dir, f"roi_image_{timestamp}.jpg")
                cv2.imwrite(roi_image_path, roi)
                print(f"ROI image saved: {roi_image_path}")

cap.release()
cv2.destroyAllWindows()

I0000 00:00:1725212552.728539 1913698 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1725212552.730840 1954873 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.1.6-arch1.1), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
W0000 00:00:1725212552.836302 1954862 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1725212552.861857 1954868 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's t

In [2]:
# Load saved ROI images
roi_dir = "saved_images"
roi_images = []
for filename in os.listdir(roi_dir):
    if filename.startswith("roi_image_"):
        img = cv2.imread(os.path.join(roi_dir, filename))
        roi_images.append(img)

# Create background subtractor
bg_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

# Process each ROI image
for i, roi in enumerate(roi_images):
    
    # Apply background subtraction
    fg_mask = bg_subtractor.apply(roi)
    
    # Post-process mask
    fg_mask = cv2.medianBlur(fg_mask, 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    
    # Get foreground
    fg = cv2.bitwise_and(roi, roi, mask=fg_mask)
    
    # Save result
    cv2.imwrite(f"fg_roi_{i}.jpg", fg)

    # Display result
    cv2.imshow("Foreground", fg)
    cv2.waitKey(0)

cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to target thread (0x5ec952b30240)

QObject::moveToThread: Current thread (0x5ec952b30240) is not the object's thread (0x5ec9533363b0).
Cannot move to tar

In [17]:
import cv2
import numpy as np
import os

# Create a directory for processed images
processed_dir = 'processed_images'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Load the ROI image
roi_image = cv2.imread('saved_images/roi_image_20240901_222255.jpg')

# Convert to grayscale
gray_roi = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur
blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

# Use adaptive thresholding
binary_roi = cv2.adaptiveThreshold(blurred_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

# Apply morphological operations
kernel = np.ones((3,3), np.uint8)
cleaned_roi = cv2.morphologyEx(binary_roi, cv2.MORPH_CLOSE, kernel, iterations=2)
cleaned_roi = cv2.morphologyEx(cleaned_roi, cv2.MORPH_OPEN, kernel, iterations=1)

# Find contours
contours, _ = cv2.findContours(cleaned_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a mask for the main object
mask = np.zeros(gray_roi.shape, np.uint8)
for contour in contours:
    if cv2.contourArea(contour) > 100:  # Adjust this threshold as needed
        cv2.drawContours(mask, [contour], 0, (255), -1)

# Apply the mask to the original grayscale image
result = cv2.bitwise_and(gray_roi, gray_roi, mask=mask)

# Invert the result
result_inverted = cv2.bitwise_not(result)

# Save intermediate and final results
cv2.imwrite(os.path.join(processed_dir, 'gray_roi.jpg'), gray_roi)
cv2.imwrite(os.path.join(processed_dir, 'binary_roi.jpg'), binary_roi)
cv2.imwrite(os.path.join(processed_dir, 'cleaned_roi.jpg'), cleaned_roi)
cv2.imwrite(os.path.join(processed_dir, 'mask.jpg'), mask)
cv2.imwrite(os.path.join(processed_dir, 'result.jpg'), result)
cv2.imwrite(os.path.join(processed_dir, 'result_inverted.jpg'), result_inverted)

print(f"Processed images saved in '{processed_dir}' directory.")

Processed images saved in 'processed_images' directory.


In [18]:
import cv2
import numpy as np
import os

# Create a directory for processed images if it doesn't exist
processed_dir = 'processed_images'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Load the ROI image
roi_image = cv2.imread('saved_images/roi_image_20240901_222255.jpg')

# Convert to grayscale
gray_roi = cv2.cvtColor(roi_image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise
blurred_roi = cv2.GaussianBlur(gray_roi, (5, 5), 0)

# Global thresholding
_, global_thresh = cv2.threshold(blurred_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Adaptive thresholding
adaptive_thresh = cv2.adaptiveThreshold(blurred_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Save the results
cv2.imwrite(os.path.join(processed_dir, 'global_thresholded.jpg'), global_thresh)
cv2.imwrite(os.path.join(processed_dir, 'adaptive_thresholded.jpg'), adaptive_thresh)

# Optional: Apply morphological operations to clean up the results
kernel = np.ones((3,3), np.uint8)
global_cleaned = cv2.morphologyEx(global_thresh, cv2.MORPH_OPEN, kernel, iterations=1)
adaptive_cleaned = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_OPEN, kernel, iterations=1)

cv2.imwrite(os.path.join(processed_dir, 'global_cleaned.jpg'), global_cleaned)
cv2.imwrite(os.path.join(processed_dir, 'adaptive_cleaned.jpg'), adaptive_cleaned)

print(f"Processed images saved in '{processed_dir}' directory.")


Processed images saved in 'processed_images' directory.


In [20]:
import cv2
import numpy as np
import os

# Create processed_images directory if it doesn't exist
processed_dir = 'processed_images'
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Load the processed result image
result_image = cv2.imread(os.path.join(processed_dir, 'result.jpg'), 0)

# Apply Sobel filters
sobelx = cv2.Sobel(result_image, cv2.CV_64F, 1, 0, ksize=3)
sobely = cv2.Sobel(result_image, cv2.CV_64F, 0, 1, ksize=3)
gradient_magnitude = np.sqrt(sobelx**2 + sobely**2)
gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

# Create markers for watershed
_, markers = cv2.threshold(gradient_magnitude, 40, 255, cv2.THRESH_BINARY)
markers = cv2.connectedComponents(markers)[1]

# Apply watershed algorithm
markers = markers.astype(np.int32)
cv2.watershed(cv2.cvtColor(result_image, cv2.COLOR_GRAY2BGR), markers)

# Create a color map for visualization
color_map = np.random.randint(0, 255, size=(np.max(markers) + 1, 3), dtype=np.uint8)
segmented_image = color_map[markers]

# Save the results
cv2.imwrite(os.path.join(processed_dir, 'gradient_magnitude.jpg'), gradient_magnitude)
cv2.imwrite(os.path.join(processed_dir, 'watershed_segmentation.jpg'), segmented_image)

print(f"Gradient and watershed segmentation results saved in '{processed_dir}' directory.")

Gradient and watershed segmentation results saved in 'processed_images' directory.


### Segmentation